## Summer 2022 Data Science Intern Challenge by Edward Kim 

#### 1. Import libraries

In [26]:
from csv import reader 
import math
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import statistics 

#### 2. Import data

In [24]:
filename = '2019 Winter Data Science Intern Challenge Data Set - Sheet1.csv'
df = pd.read_csv(filename)

#### 3. Initial check for missing data

In [22]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   order_id        5000 non-null   int64 
 1   shop_id         5000 non-null   int64 
 2   user_id         5000 non-null   int64 
 3   order_amount    5000 non-null   int64 
 4   total_items     5000 non-null   int64 
 5   payment_method  5000 non-null   object
 6   created_at      5000 non-null   object
dtypes: int64(5), object(2)
memory usage: 273.6+ KB
None


The output shows that there are 5,000 entries and no null values. 

#### 4. Reproduction of AOV Calculation Error in Previous Model and Re-evaluation of AOV

In [6]:
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


##### 4.a Reproduction of AOV Calculation Error in Previous Model

In [7]:
# Reproduce AOV calculation error 
oa_sum = df['order_amount'].sum()
ti_count = df['total_items'].count()
aov = round(oa_sum/ti_count, 2)
print(aov)

3145.13


Using .count() on total_items column returns 5,000 which is the number of rows in the csv file. 5,000 is less than the actual value of total_items sold as some rows contain > 1 items sold. Thus AOV was wrontly defined as the division of the total order amount by the number of rows. 

##### 4.b Re-evaluate AOV with corrected total_items count

In [9]:
# Return the actual AOV by changing ti_count from the number of rows to the sum of values in each order. 
ti_count = df['total_items'].sum()
aov = round(oa_sum/ti_count, 2)
print(aov)

357.92


By changing .count() to .sum(), the average order value decreases to 357.92 which is a significantly more reasonable price than 3145.13 for shoes. 

#### 5. Identifications of outliers for better metric

In [20]:
df.describe().round(2)

,order_id,shop_id,user_id,order_amount,total_items
count,5000.00,5000.00,5000.00,5000.00,5000.00
mean,2500.50,50.08,849.09,3145.13,8.79
std,1443.52,29.01,87.80,41282.54,116.32
min,1.00,1.00,607.00,90.00,1.00
25%,1250.75,24.00,775.00,163.00,1.00
50%,2500.50,50.00,849.00,284.00,2.00
75%,3750.25,75.00,925.00,390.00,3.00
max,5000.00,100.00,999.00,704000.00,2000.00


The std for order_amount does not seem reasonable, therefore, there is likely and outlier in the data. The large variation in total_items is ignored since some buyers may have conducted a large order. The more important figure is the order_amount. To determine if the large variation in order amount is caused by a large number of total_items, the order_amount can be configured to represent a single item. 

In [84]:
df_copy = df.copy()
df_copy = df_copy[['shop_id', 'user_id', 'order_amount', 'total_items']]

df_copy['single_order_amount'] = df_copy['order_amount']/df_copy['total_items']

df_copy.groupby(df_copy['shop_id'])['single_order_amount'].mean().reset_index().sort_values(by=['single_order_amount'], ascending = False).head()


,shop_id,single_order_amount
77,78,25725.0
41,42,352.0
11,12,201.0
88,89,196.0
98,99,195.0


Output shows that shop_id=78 has a price of 25,725 for a single item. Although such value may be due to shop_id=78 being a luxury provider, ignoring the data would provide a more accurate pool of data for general shoes. 

In [101]:
oa_sum = df_copy['order_amount'].sum()
ti_count = df_copy['total_items'].sum()
aov = round(oa_sum/ti_count, 2)
print(aov)

307.01


After removing shop_id=78, an outlier, aov decreases to a more reasonable value. 

In [100]:
aov = np.mean(df_copy[df_copy['shop_id'] != 78]['single_order_amount'])
print(aov)

152.47557529269278


If all stores only sold one pair of shoes, the aov would be 152.48, which is a reasonable price for a pair of shoes. 